# parcels regions

runs parcels on existing netcdf files

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import timedelta
import math
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from parcels import FieldSet, ParticleSet
import scipy.io
import xarray as xr

import utils
from parcels_utils import get_file_info
from plot_utils import plot_particles_age

# ignore annoying deprecation warnings
import warnings
warnings.simplefilter("ignore", UserWarning)
import cartopy

# ignore divide by nan error that happens constantly with parcels
np.seterr(divide='ignore', invalid='ignore')

## configuration

change the contents of `configs` for the simulation configuration you want

In [ ]:
configs = [
    # "plume_track.json",
    # "plume_track_totsdlj.json",
#     "tijuana_interped.json",
#     "tijuana_lin.json",
#     "tijuana_lin_aggr.json",
#     "tijuana_less.json",
#     "tijuana_now.json",
#     "tijuana_onerep.json",
#     "tijuana_range.json",
    "buoy_track.json"
]

loaded_configs = [utils.load_config(utils.PARCELS_CONFIGS_DIR / path) for path in configs]
files = [get_file_info(cfg["netcdf_path"], cfg["resolution"], name=cfg["name"], parcels_cfg=cfg["parcels_config"]) for cfg in loaded_configs]

## Animated gif stuff and particle simulation

In [ ]:
# animation man very cool
# reference tutorial_Agulhasparticles
# needs ErrorCode for particle recovery
from operator import attrgetter
from parcels import ErrorCode, JITParticle, Variable, AdvectionRK4

max_v = 0.6 # for display purposes only, so the vector field colors don't change every iteration

class ThreddsParticle(JITParticle):
    lifetime = Variable("lifetime", initial=0, dtype=np.float32)
    spawntime = Variable("spawntime", initial=attrgetter("time"), dtype=np.float32)
    # out of bounds
    oob = Variable("oob", initial=0, dtype=np.int32)
    
    
def AgeParticle(particle, fieldset, time):
    particle.lifetime += particle.dt
    
    
def TestOOB(particle, fieldset, time):
    u, v = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
    if math.fabs(u) < 1e-14 and math.fabs(v) < 1e-14:
        particle.oob = 1
    else:
        particle.oob = 0


def DeleteParticle(particle, fieldset, time):
    print("Particle [%d] lost (%g %g %g %g)" % (particle.id, particle.time, particle.depth, particle.lat, particle.lon), file=sys.stderr)
    particle.delete()


def exec_pset(data, runtime, dt):
    if len(data["pset"]) == 0:
        print("ParticleSet is empty. Passing...", file=sys.stderr)
        return
    
    # temporary - TODO make it only init once
    k_age = data["pset"].Kernel(AgeParticle)
    k_oob = data["pset"].Kernel(TestOOB)

    data["pset"].execute(
        AdvectionRK4 + k_age + k_oob,
        runtime=timedelta(seconds=runtime),
        dt=timedelta(seconds=dt),
        recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
        output_file=data["pfile"]
    )


def save_pset_plot(data, i, zpad=3, ages=True):
    days = (data["timerng"][1] - data["timerng"][0]) / np.timedelta64(1, 'D')
    field = None if ages else "vector"
    if data["cfg"]["shown_domain"] is not None:
        dom = data["cfg"]["shown_domain"]
    else:
        dom = data["domain"]
    plot_particles_age(data["pset"], dom, field=field,
                       savefile=str(data["snap_path"])+"/particles"+str(i).zfill(zpad)+".png",
                       vmax=days, field_vmax=max_v, part_size=4)
        
        
def parse_time_range(time_range, data):
    """
    Args:
        time_range (array-like): some array with 2 strings
        data (dict)
    """
    if time_range[0] == "START":
        t_start = data["timerng"][0]
    else:
        try:
            t_start = int(time_range[0])
        except ValueError:
            t_start = np.datetime64(time_range[0])

    if time_range[1] == "END":
        t_end = data["timerng"][1]
    else:
        try:
            t_end = int(time_range[1])
        except ValueError:
            t_end = np.datetime64(time_range[1])
            
    if isinstance(t_start, int) and isinstance(t_end, int):
        raise TypeError("Must have at least one date in the time range.")
    if isinstance(t_start, int):
        t_start = t_end - np.timedelta64(t_start)
    if isinstance(t_end, int):
        t_end = t_start + np.timedelta64(t_end)
        
    return t_start, t_end

### ParticleSet and spawn point setup

note about interpolation methods: only `linear` works if you want to use the FieldSet in a ParticleSet.

In [ ]:
part_path = utils.create_path(utils.PARTICLE_NETCDF_DIR)

for f in files:
    cfg = f["cfg"]
    # data is from .mat file
    if isinstance(cfg["spawn_points"], str):
        spawns = utils.load_pts_mat(cfg["spawn_points"], "yf", "xf")
        cfg["spawn_points"] = spawns.T
    else:
        cfg["spawn_points"] = np.array(cfg["spawn_points"])
    
    # parse time information
    t_start, t_end = parse_time_range(cfg["time_range"], f)
    # convert to seconds relative to fieldset delta
    t_start = (t_start - f["timerng"][0]) / np.timedelta64(1, "s")
    t_end = (t_end - f["timerng"][0]) / np.timedelta64(1, "s")
            
    if cfg["repeat_dt"] <= 0:
        repetitions = 1
    else:
        repetitions = int((t_end - t_start) / cfg["repeat_dt"])
    cfg["sim_start_sec"] = t_start
    cfg["sim_end_sec"] = t_end
    # the total number of particles that will exist in the simulation
    if cfg["particles_per_dt"] <= 0:
        cfg["particles_per_dt"] = len(cfg["spawn_points"])
        total = repetitions * len(cfg["spawn_points"])
    else:
        total = repetitions * cfg["particles_per_dt"]

    # prep when each particle is released
    time_arr = np.zeros(total)
    for i in range(repetitions):
        # should be fine if repeat_dt is negative since it will be multiplied by 0
        # when that's the case
        time_arr[cfg["particles_per_dt"] * i:cfg["particles_per_dt"] * (i + 1)] = t_start + cfg["repeat_dt"] * i

    # select spawn points from the given config
    if cfg["random_spawn"]:
        # randomly choose spawn points and random vairation
        sp_lat = cfg["spawn_points"].T[0][np.random.randint(0, len(cfg["spawn_points"]), total)]
        sp_lon = cfg["spawn_points"].T[1][np.random.randint(0, len(cfg["spawn_points"]), total)]
        # vary spawn locations using max_variation
        p_lats = utils.add_noise(sp_lat, cfg["max_variation"])
        p_lons = utils.add_noise(sp_lon, cfg["max_variation"])
    else:
        # cycle through each spawn point in order
        sp_repeat = int(total / len(cfg["spawn_points"]))
        remainder = total - sp_repeat * len(cfg["spawn_points"])
        p_lats = np.empty(total)
        p_lons = np.empty(total)
        p_lats[:sp_repeat * len(cfg["spawn_points"])] = np.tile(cfg["spawn_points"].T[0], sp_repeat)
        p_lats[sp_repeat * len(cfg["spawn_points"]):] = cfg["spawn_points"].T[0][:remainder]
        p_lons[:sp_repeat * len(cfg["spawn_points"])] = np.tile(cfg["spawn_points"].T[1], sp_repeat)
        p_lons[sp_repeat * len(cfg["spawn_points"]):] = cfg["spawn_points"].T[1][:remainder]

    # set up ParticleSet and ParticleFile
    f["pset"] = ParticleSet(fieldset=f["fs"], pclass=ThreddsParticle, lon=p_lons, lat=p_lats, time=time_arr)
    save_path = part_path / f"particle_{f['name']}.nc"
    f["pfile"] = f["pset"].ParticleFile(save_path)
    print(f"Particle trajectories for {f['name']} will be saved to {save_path}")
    print(f"    total particles in simulation: {total}")

In [ ]:
[f["pset"].show(field="vector", vmax=max_v) for f in files]

### simulation setup and execution

simulation parameter setup

In [ ]:
# setting up times, intervals, and paths for simulation
for f in files:
    cfg = f["cfg"]
    cfg["snap_num"] = math.floor((cfg["sim_end_sec"] - cfg["sim_start_sec"]) / cfg["snapshot_interval"])
    cfg["last_int"] = cfg["sim_end_sec"] - (cfg["snap_num"] * cfg["snapshot_interval"] + cfg["sim_start_sec"])
    if cfg["last_int"] == 0:
        print("No last interval exists.")
        print(f"Num snapshots to save for {f['path']}: {cfg['snap_num'] + 1}")
    else:
        print(f"Num snapshots to save for {f['path']}: {cfg['snap_num'] + 2}")
    if cfg["snap_num"] >= 200 and cfg["save_snapshots"]:
        raise Exception(f"Too many snapshots ({cfg['snap_num']}).")
    f["snap_path"] = utils.create_path(utils.PICUTRE_DIR / f"{utils.filename_dict[f['res']]}/{f['name']}")
    print(f"Path to save snapshots to: {f['snap_path']}")
    # only clear directory if desired or actually saving images
    if cfg["save_snapshots"]:
        for p in f["snap_path"].glob("*.png"):
            p.unlink()

execution of all simulation configurations

In [ ]:
# show age or not (temporary)
age = False
# execution of simulation
for f in files:
    cfg = f["cfg"]
    if cfg["save_snapshots"]:
        save_pset_plot(f, 0, ages=age)
    for i in range(cfg["snap_num"]):
        exec_pset(f, cfg["snapshot_interval"], cfg["simulation_dt"])
        if cfg["save_snapshots"]:
            save_pset_plot(f, i + 1, ages=age)

    # save the second-to-last frame
    if cfg["save_snapshots"]:
        save_pset_plot(f, cfg["snap_num"] + 1, ages=age)

    # run the last interval (the remainder) if needed
    if cfg["last_int"] != 0:
        exec_pset(f, cfg["snapshot_interval"], cfg["simulation_dt"])
        if cfg["save_snapshots"]:
            save_pset_plot(f, cfg["snap_num"] + 2, ages=age)
        
    # run one more time with 0 runtime because for some reason
    # particles can be out-of-bounds without being deleted
    # ??? how does this happen ???
    # im not actually sure if this really does anything but hey its worth a shot
    exec_pset(f, 0, 0)
        
    f["pfile"].export()
    f["pfile"].close()

print("all simulations done and snapshots saved (if simulation was saving snapshots)")

### gif generation

don't have to run, requires [magick](https://imagemagick.org/index.php)

the gifs will be saved `snapshots/west_coast_xkm_hourly/` where xkm is the resolution

In [ ]:
import subprocess

gif_delay = 25 # ms

for f in files:
    if f["cfg"]["save_snapshots"]:
        utils.create_gif(
            gif_delay,
            str(f["snap_path"]) + "/*.png",
            utils.PICUTRE_DIR / f"{utils.filename_dict[f['res']]}/partsim_{f['name']}.gif"
        )
    else:
        print("no gif generated")